In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itard"
fs_method, fs_ratio = "dnp", 0.5
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

Number of Selected Features: 100%|██████████| 71/71 [07:00<00:00,  5.92s/it]


In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


Epoch 1/50:  46%|████▌     | 12/26 [00:00<00:00, 115.11batch/s, loss=0.263]

model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9156 | AUPR: 0.6550 | Acc: 95.45%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9192 | AUPR: 0.6620 | Acc: 95.34%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9223 | AUPR: 0.6705 | Acc: 95.37%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9165 | AUPR: 0.6556 | Acc: 95.43%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9200 | AUPR: 0.6617 | Acc: 95.39%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9225 | AUPR: 0.6675 | Acc: 95.44%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9225 | AUPR: 0.6675 | Acc: 95.44%
Internal test: AUC: 0.9275 | AUPR: 0.7019 | Acc: 95.33%
External test: AUC: 0.9224 | AUPR: 0.6728 | Acc: 95.26%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9182 | AUPR: 0.6249 | Acc: 94.99%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9176 | AUPR: 0.6177 | Acc: 94.95%
model(kernel=linear, C=1)
Valid:         AUC: 0.9167 | AUPR: 0.6148 | Acc: 94.93%
best model(kernel=linear, C=0.01)
Valid:         AUC: 0.9182 | AUPR: 0.6249 | Acc: 94.99%
Internal test: AUC: 0.9233 | AUPR: 0.6633 | Acc: 94.99%
External test: AUC: 0.9178 | AUPR: 0.6125 | Acc: 94.64%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7584 | AUPR: 0.4149 | Acc: 94.66%
model(n_neighbors=5)
Valid:         AUC: 0.7914 | AUPR: 0.4732 | Acc: 94.96%
model(n_neighbors=7)
Valid:         AUC: 0.8116 | AUPR: 0.5039 | Acc: 94.96%
best model(n_neighbors=7)
Valid:         AUC: 0.8116 | AUPR: 0.5039 | Acc: 94.96%
Internal test: AUC: 0.8022 | AUPR: 0.5174 | Acc: 94.77%
External test: AUC: 0.8103 | AUPR: 0.5027 | Acc: 94.83%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8765 | AUPR: 0.4550 | Acc: 83.58%
best model()
Valid:         AUC: 0.8765 | AUPR: 0.4550 | Acc: 83.58%
Internal test: AUC: 0.8892 | AUPR: 0.5034 | Acc: 84.18%
External test: AUC: 0.8759 | AUPR: 0.4456 | Acc: 82.47%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9187 | AUPR: 0.6544 | Acc: 95.28%
model(C=10.0)
Valid:         AUC: 0.9184 | AUPR: 0.6534 | Acc: 95.29%
model(C=100.0)
Valid:         AUC: 0.9183 | AUPR: 0.6532 | Acc: 95.29%
best model(C=1.0)
Valid:         AUC: 0.9187 | AUPR: 0.6544 | Acc: 95.28%
Internal test: AUC: 0.9234 | AUPR: 0.6873 | Acc: 95.26%
External test: AUC: 0.9148 | AUPR: 0.6372 | Acc: 94.97%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8828 | AUPR: 0.5506 | Acc: 83.45%
model(criterion=log_loss)
Valid:         AUC: 0.8687 | AUPR: 0.5650 | Acc: 82.33%
model(criterion=entropy)
Valid:         AUC: 0.8687 | AUPR: 0.5650 | Acc: 82.33%
best model(criterion=gini)
Valid:         AUC: 0.8828 | AUPR: 0.5506 | Acc: 83.45%
Internal test: AUC: 0.8690 | AUPR: 0.5973 | Acc: 85.74%
External test: AUC: 0.8752 | AUPR: 0.5573 | Acc: 85.46%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9163 | AUPR: 0.6565 | Acc: 89.93%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9172 | AUPR: 0.6587 | Acc: 89.95%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9176 | AUPR: 0.6591 | Acc: 90.09%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9156 | AUPR: 0.6558 | Acc: 90.00%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9164 | AUPR: 0.6577 | Acc: 89.96%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9168 | AUPR: 0.6596 | Acc: 90.13%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9156 | AUPR: 0.6558 | Acc: 90.00%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9164 | AUPR: 0.6577 | Acc: 89.96%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9168 | AUPR: 0.6596 | Acc: 90.13%
best model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9176 | AUPR: 0.6591 | Acc: 9

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9215 | AUPR: 0.6520 | Acc: 95.16%
model(criterion=squared_error)
Valid:         AUC: 0.9216 | AUPR: 0.6526 | Acc: 95.17%
best model(criterion=squared_error)
Valid:         AUC: 0.9216 | AUPR: 0.6526 | Acc: 95.17%
Internal test: AUC: 0.9243 | AUPR: 0.6936 | Acc: 95.28%
External test: AUC: 0.9217 | AUPR: 0.6573 | Acc: 95.05%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9190 | AUPR: 0.6471 | Acc: 95.33%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9181 | AUPR: 0.6457 | Acc: 95.35%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9187 | AUPR: 0.6479 | Acc: 95.28%
best model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9190 | AUPR: 0.6471 | Acc: 95.33%
Internal test: AUC: 0.9206 | AUPR: 0.6700 | Acc: 95.16%
External test: AUC: 0.9142 | AUPR: 0.6245 | Acc: 94.94%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)